## 0. Environment Setup
We create a smiliar environemnt that simulate the inputs and variables as per in ciop jobs

In [199]:
#old input1='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7' #niger-hype-model.zip, one file when unzipped
#old input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
input2='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' #niger-hype-model-2.23.zip, one dir when unzipped
#use this from model.zip instead  input_hype_data='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2' #niger-hype-data.zip, dirs when unzipped
TMPDIR="/workspace/tmpdir"
TMPDIR

[1] "/workspace/tmpdir"

1. Read input model
This inital step collect via proper opensearch call and copy the HYPE model config archive

In [200]:
app.date = format(Sys.time(), "%Y%m%d_%H%M")

## set application name
app.name = "forecast"
## ------------------------------------------------------------------------------
## flag which environment is used, if not set
if(!exists("app.sys")){
    app.sys ="tep"
    }
## ------------------------------------------------------------------------------
## load rciop package and set working directory to TMPDIR when running on TEP 
if(app.sys=="tep"){
    library("rciop")

    rciop.log ("DEBUG", " *** hypeapps-forecast *** TEP hydrological modelling applications ***", "/node_forecast/run.R")
    rciop.log ("DEBUG", " rciop library loaded", "/node_forecast/run.R")

    setwd(TMPDIR)
    rciop.log("DEBUG", paste(" R session working directory set to ",TMPDIR,sep=""), "/node_forecast/run.R")
}

Some rciop functions may need to be overridden in the notebook environment.
Set to your current needs.

In [201]:
ENABLE_STUB_RCIOP_LOG <- FALSE
#ENABLE_STUB_RCIOP_LOG <- TRUE

if (ENABLE_STUB_RCIOP_LOG == TRUE) {
    message("rciop.log() overridden by local stub function...")
    
    rciop.log <- function(type, message, proc='') {

        type <- type
        msg <- message
    if (proc=='') { proc <- "user process" }

    out.msg <- paste(format(Sys.time(), format="%Y-%m-%dT%H:%M:%S"), "[", type, "][", proc, "] ", msg, sep="")

    message(out.msg)
    } # rciop.log

} # end stub

In [202]:
#ENABLE_STUB_RCIOP_GETPARAM <- FALSE
ENABLE_STUB_RCIOP_GETPARAM <- TRUE

if (ENABLE_STUB_RCIOP_GETPARAM == TRUE) {
    message("rciop.getparam() overridden by local stub function...")
    # Change according to your test objective.
    # Would normally return values defined in application.xml for specified ids'.
    
    rciop.getparam <- function(id) {
        value <- NULL # or ""
        
        if (id == "model") {
            value <- "https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7"
        }
        else if (id == "idate" ) {
            value <- "2018-09-01"
        }
        else if (id == "hydmodel" ) {
            c()
            #value[1] <- "Niger-HYPE"
            #value[2] <- "WestAfrica-HYPE"
            value <- c('Niger-HYPE','WestAfrica-HYPE')
        }
        else {
            message("rciop.get_param:id not found...")
        }
        
        return (value)
    } # rciop.getparam

} # end stub

rciop.getparam() overridden by local stub function...


Add your code to test...
Remove current, or paste other code or add additional code blocks.

In [26]:
## Read the main input
## This is the reference link to the model configuration

# print
#rciop.log("INFO", paste("Processing input:", input1, sep=" "))

# Query the input reference
#message(paste("opensearch-client '",input1,"' enclosure", sep=""))
#opensearchCmd=paste("opensearch-client '",input1,"' enclosure")
#input_enclosure <- system(command = opensearchCmd,intern = T)
#rciop.log("INFO", input_enclosure)

# Download the file
#model_file <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

#if (model_file$exit.code==0) {
#    local.url <- model_file$output
#}

# Works for single file after unzip
#local.url # One file, complete path incl. filename
#my_data <- read.delim(local.url)
#print (my_data)

In [27]:
#par_model <- rciop.getparam("model")
#message(par_model)

In [203]:
## Read the main input
## This is the reference link to the model configuration

# print
rciop.log("INFO", paste("Processing input:", input2, sep=" "))

# Query the input reference
message(paste("opensearch-client '",input2,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",input2,"' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", input_enclosure)

# Download the dir(s)
model_config_dir <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

if (model_config_dir$exit.code==0) {
    local.model_config_dir <- model_config_dir$output
}
local.model_config_dir # File or directory, complete path incl.
# Content for now:
# info-forecast.txt - config file for Hype model

#if (local.model_config_dir == "" || local.model_config_dir == NULL) {
#    message("No model config path...")
#    q(99)
#}

model_config_file <- "dependencies.txt" # Shall be parsed to get different config items
# tmpdir/niger-hype-model-2.23/ # Currently for now:
# - dependencies.txt # Config for GFD-files etc.
# - info-forecast.txt # Hype config file


path_to_file <- paste(local.model_config_dir, model_config_file, sep="/")
path_to_file
# Read contents of file, handling separators ';'.
model_config_data <- read.csv2(path_to_file, header=FALSE, sep=";")
#class(model_config_data) # data.frame
#dim(model_config_data) # 6,3 (rows,cols)
# Add column names
names(model_config_data) <- c('subdir','url','querypattern','info')
model_config_data
# Select column by name
#model_config_data[,'query-pattern'] # all rows, col 3 ok
#model_config_data[,c('subdir','querypattern')] # all rows, multiple columns ok

#model_config_data$subdir
#model_config_data[3,'subdir']
# Iterate over each row to locate e.g. sub directory, od-daily, to use column data
for (r in 1:nrow(model_config_data)) { #  if statement below not works
    subdir <- model_config_data[r,'subdir']
    if (subdir == 'od-daily') {
        message(paste0("od-daily at row index:", r))
    }
    if (subdir == 'model-data') {
        message(paste0("model-data at row index:", r))
    }
}

opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9345ED73B72F49E6FF31B07B57013BC519210E24' enclosure


[1] " -o /workspace/tmpdir https://store.terradue.com/hydro-smhi/models/niger-hype-model-2.23.zip"


[1] "/workspace/tmpdir/niger-hype-model-2.23"

[1] "/workspace/tmpdir/niger-hype-model-2.23/dependencies.txt"

subdir,url,querypattern,info
model-data,https://recast.terradue.com/t2api/search/hydro-smhi/models,?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2,# zip-file. niger-hype v2.23 and gfd obs.
model-data-old-url,https://store.terradue.com/hydro-smhi/fanfar/model/niger-hype,,# stop using. not zip-file. niger-hype v2.23 and gfd obs.
hydrogfdei,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdei,SMHI,operational]",# netcdf
hydrogfdod,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdod,SMHI,operational]",# netcdf
ei-monthly,https://catalog.terradue.com/hydro-smhi/description,"[ei,monthly,SMHI,operational]",# netcdf
od-daily,https://catalog.terradue.com/hydro-smhi/description,"[od,daily,SMHI,operational]",# netcdf
od-monthly,https://catalog.terradue.com/hydro-smhi/description,"[od,monthly,SMHI,operational]",# netcdf
ecoper,https://catalog.terradue.com/hydro-smhi/description,"[ecoper,SMHI,operational]",# netcdf


model-data at row index:1
od-daily at row index:6


In [204]:
## Read the Hype data dirs
## This code continues to retrieve the data dirs (rciop.copy)
## Other R code not yet updated to handle paths (file copy from tmp dir to run dir etc.) instead of urls.

# print
#rciop.log("INFO", paste("Processing config for model data:", input2, sep=" "))
rciop.log("INFO", "Processing config for model data")

rowIndex=0
for (r in 1:nrow(model_config_data)) {
    subdir <- model_config_data[r,'subdir']
    if (subdir == 'model-data') {
        rowIndex=r
        message(paste0("model-data at row index:", r))
    }
}
if (rowIndex == 0) {
    q(98)
}
#subdir <- model_config_data[rowIndex,'subdir']
url <- model_config_data[rowIndex,'url']
query <- model_config_data[rowIndex,'querypattern']
#comment <- model_config_data[rowIndex,'info']

# For now, if needed, pass url from model_config_data['model-data-old-url'] to let present R code continue rciop.copy locally.

# Query the input reference
message(paste("opensearch-client '",url,query,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",url,query,"' enclosure")
input_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", input_enclosure)

# Download the dir(s)
model_data_dirs <- rciop.copy(input_enclosure, TMPDIR, uncompress=TRUE)

if (model_data_dirs$exit.code==0) {
    local.dir <- model_data_dirs$output
}
local.dir # File or directory, complete path incl.
# tmpdir/niger-hype-data/
# - data/gfd/archive/Pobs.txt, TMAXobs.txt, TMINobs.txt, Tobs.txt
# - v2.23/....... root dir for forcingarchive, shapefiles, statefiles etc.
# This/These path(s) shall be passed to getHypeAppSetup as dir, now urls that it currently downloads - now already downloaded...
# No file to parse etc. only pass path to the R-get functions


model-data at row index:1
opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=9CE420998D9EC367470EEAEEA6B6DA0B0E4604D2' enclosure


[1] " -o /workspace/tmpdir https://store.terradue.com/hydro-smhi/models/niger-hype-data.zip"


[1] "/workspace/tmpdir/niger-hype-data"

In [54]:
# opensearch for netcdf files:


In [228]:
# Query the input reference
#opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure,identifier")
# Gives a lot of urls to files, test with e.g. count=4 (for the 4 lastest?)

# When time to perform the download, only use enclosure skip identifier etc.
#opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure")
# # count=4 (for the 4 lastest, yes) gives pr,tas,tasmin and tasmax
#opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=1' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure")
# # count=1 gives me pr at this time, maybe the lastest time
#opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=1' -p 'cat=[SMHI,operational,daily,od,tas]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure")
# # count=1 and tas specified gives me tas - nice

#
# Test different variant of indata
#
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od,tas]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure")
# With unlimited and no time stamp we receive tas_od-daily_20190301_fanfar_SMHI.nc and tas_od-daily_20190228_fanfar_SMHI.nc - partial ok

opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od,tas]' -p 'start=2019-03-01T00:00:01' -p 'stop=2019-03-01T23:59:59' enclosure")
# With unlimited and time stamp we receive tas_od-daily_20190301_fanfar_SMHI.nc only - ok

# Check all variables for daily-od with count 4
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-03-01T00:00:01' -p 'stop=2019-03-01T23:59:59' enclosure")
# ok

# Check all variables for daily-od with count, cat and different date/time intervals (timestamp to not get the previous day in results)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,daily,od]' -p 'start=2019-04-15T00:00:01' -p 'stop=2019-04-15T23:59:59' enclosure") # ok, files for 20190415 (pr,tas,tasmin,tasmax)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od,pr]' -p 'start=2019-04-01T00:00:01' -p 'stop=2019-04-01T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od,tasmin]' -p 'start=2019-04-02T00:00:01' -p 'stop=2019-04-02T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od,tasmax]' -p 'start=2019-04-03T00:00:01' -p 'stop=2019-04-03T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,od,tas]' -p 'start=2019-04-05T00:00:01' -p 'stop=2019-04-05T23:59:59' enclosure") # 2019-04-04 did not exist at smhi # 05 ok

# Check all variables for ecoper with count, cat and different date/time intervals (timestamp to not get the previous day in results)
# cat=daily maybe only intended for gfd files in dir od-daily...
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,ecoper]' -p 'start=2019-05-20T00:00:01' -p 'stop=2019-05-20T23:59:59' enclosure") # nok, nothing
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ecoper]' -p 'start=2019-05-20T00:00:01' -p 'stop=2019-05-20T23:59:59' enclosure") # nok, everything 2017..2019
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,daily,ecoper]' -p 'start=2019-05-20' -p 'stop=2019-05-20' enclosure") # nok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ecoper]' -p 'start=2019-05-20' -p 'stop=2019-05-20' enclosure") # nok, everything 2017..2019
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ecoper]' -p 'start=2019052000' -p 'stop=2019052000' enclosure") # filename style, nok, everything 2017..2019
# Try ecoper without SMHI and operational, ignore daily, only intended for od-daily)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ecoper]' -p 'start=2019-05-20T00:00:01' -p 'stop=2019-05-20T23:59:59' enclosure") # nok, everything 2017..2019
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[operational,ecoper]' -p 'start=2019-05-20T00:00:01' -p 'stop=2019-05-20T23:59:59' enclosure") # nok, everything 2017..2019
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,ecoper]' -p 'start=2019-05-20T00:00:01' -p 'stop=2019-05-20T23:59:59' enclosure") # nok, everything 2017..2019
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[ecoper]' -p 'start=2019-05-20T00:00:01' -p 'stop=2019-05-20T23:59:59' enclosure") # nok, everything 2017..2019

# Check all variables for hydrogfdod with count, cat and different date/time intervals
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,hydrogfdod]' -p 'start=2019-06-13T00:00:01' -p 'stop=2019-06-13T23:59:59' enclosure") # ok, files for 201906 (pr,tas,tasmin,tasmax)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,hydrogfdod]' -p 'start=2019-07-01T00:00:01' -p 'stop=2019-07-31T23:59:59' enclosure") # ok, files for 201907 (pr,tas,tasmin,tasmax)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdod,pr]' -p 'start=2018-03-01T00:00:01' -p 'stop=2018-03-31T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdod,tas]' -p 'start=2018-02-11T00:00:01' -p 'stop=2018-02-11T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdod,tasmin]' -p 'start=2017-01-11T00:00:01' -p 'stop=2017-01-11T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdod,tasmax]' -p 'start=2016-11-20T00:00:01' -p 'stop=2016-11-20T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,monthly,hydrogfdod,tasmax]' -p 'start=2016-11-20T00:00:01' -p 'stop=2016-11-20T23:59:59' enclosure") # ok without cat=monthly

# Check all variables for hydrogfdei with count, cat and different date/time intervals
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,hydrogfdei]' -p 'start=2019-02-07T00:00:01' -p 'stop=2019-02-07T23:59:59' enclosure") # ok, files for 201902 (pr,tas,tasmin,tasmax)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=4' -p 'cat=[SMHI,operational,hydrogfdei]' -p 'start=2019-01-01T00:00:01' -p 'stop=2019-01-01T23:59:59' enclosure") # ok, files for 201901 (pr,tas,tasmin,tasmax)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdei,pr]' -p 'start=2018-01-01T00:00:01' -p 'stop=2018-01-31T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdei,tas]' -p 'start=2017-01-01T00:00:01' -p 'stop=2017-01-31T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdei,tasmin]' -p 'start=2017-11-01T00:00:01' -p 'stop=2017-11-30T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdei,tasmax]' -p 'start=2016-12-01T00:00:01' -p 'stop=2016-12-31T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,monthly,hydrogfdei,tasmax]' -p 'start=2016-12-01T00:00:01' -p 'stop=2016-12-31T23:59:59' enclosure") # ok without cat=monthly

# Check all variables for ei-monthly with count, cat and different date/time intervals
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=2' -p 'cat=[SMHI,operational,ei,monthly]' -p 'start=1997-01-01T00:00:01' -p 'stop=1997-01-31T23:59:59' enclosure") # ok, files for 199701 (pr,tas)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=2' -p 'cat=[SMHI,operational,ei,monthly]' -p 'start=1998-02' -p 'stop=1998-02' enclosure") # ok, files for 199802 (pr,tas)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ei,monthly,pr]' -p 'start=2004-03-01T00:00:01' -p 'stop=2004-03-31T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ei,monthly,tas]' -p 'start=2007-11-01T00:00:01' -p 'stop=2007-11-30T23:59:59' enclosure") # ok

# Check all variables for od-monthly with count, cat and different date/time intervals
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=2' -p 'cat=[SMHI,operational,od,monthly]' -p 'start=2016-08-01T00:00:01' -p 'stop=2016-08-31T23:59:59' enclosure") # ok, files for 201608 (pr,tas)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=2' -p 'cat=[SMHI,operational,od,monthly]' -p 'start=2017-10' -p 'stop=2017-10' enclosure") # ok, files for 201710 (pr,tas)
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,od,monthly,pr]' -p 'start=2017-12-01T00:00:01' -p 'stop=2017-12-31T23:59:59' enclosure") # ok
opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,od,monthly,tas]' -p 'start=2019-08-01T00:00:01' -p 'stop=2019-08-31T23:59:59' enclosure") # ok


# 2019-10-07:
#ok 201901 opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ei,monthly]' -p 'start=2019-01-02T00:00:01' -p 'stop=2019-01-03T23:59:59' enclosure")
#ok 201901 opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,od,monthly]' -p 'start=2019-01-02T00:00:01' -p 'stop=2019-01-03T23:59:59' enclosure")
#ok 20190102 and 20190103 opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,od,daily]' -p 'start=2019-01-02T00:00:01' -p 'stop=2019-01-03T23:59:59' enclosure")
#ok 201901 opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdei]' -p 'start=2019-01-02T00:00:01' -p 'stop=2019-01-03T23:59:59' enclosure")
#ok 201901 opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,hydrogfdod]' -p 'start=2019-01-02T00:00:01' -p 'stop=2019-01-03T23:59:59' enclosure")
#nok "all available dates" opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,ecoper]' -p 'start=2019-01-02T00:00:01' -p 'stop=2019-01-03T23:59:59' enclosure")

# Test with two cat parameters
#nok opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[SMHI,operational,od,monthly]' -p 'cat=tas' -p 'start=2019-08-01T00:00:01' -p 'stop=2019-08-31T23:59:59' enclosure") # ok

#
# Search
#
input_enclosure <- system(command = opensearchCmd,intern = T)
# Print
message(input_enclosure)

if (length(input_enclosure >= 1)) {
    nc_tmpdir <- paste(TMPDIR,"nc-files",sep="/")
    if (! dir.exists(nc_tmpdir)) { dir.create(nc_tmpdir) }
    for (url in 1:length(input_enclosure)) {
        rciop.copy(input_enclosure[url],nc_tmpdir)
    }
}

https://store.terradue.com/hydro-smhi/fanfar/Operational/od-monthly/2019/08/tas_od-monthly_201908_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-monthly/2019/08/pr_od-monthly_201908_fanfar_SMHI.nc


[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-monthly/2019/08/tas_od-monthly_201908_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/nc-files https://store.terradue.com/hydro-smhi/fanfar/Operational/od-monthly/2019/08/pr_od-monthly_201908_fanfar_SMHI.nc"


In [205]:
products_monthly <- c('hydrogfdei','hydrogfdod')
products_daily <- c('od-daily','ecoper')
#variables <- c('pr','tas','tasmin','tasmax')
#dates_monthly="2019-03-01" #"2019-03"
#dates_daily="2019-04-01"

#for (p in products_daily) {
#for (v in variables) {
##paste...
#opensearchCmd=paste("opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=1' -p 'cat=[SMHI,operational,daily,od,tas]' -p 'start=2019-03-01' -p 'stop=2019-03-01' enclosure")
#}
#}

In [217]:
# From process-netcdf.R
# Fake some data

netcdfDirRoot <- paste(TMPDIR,"process-nc",sep="/")

osClientApp <- "opensearch-client"
secStartDay <- "T00:00:01"
secEndDay   <- "T23:59:59"

modelConfig <- list("gfdHydrogfdeiSubDir"="hydrogfdei",
                    "gfdHydrogfdeiUrl"="https://catalog.terradue.com/hydro-smhi/description",
                    "gfdHydrogfdeiQuery"="[hydrogfdei,SMHI,operational]",
                    "gfdHydrogfdeiComment"="",
                            
                    "gfdHydrogfdodSubDir"="hydrogfdod",
                    "gfdHydrogfdodUrl"="https://catalog.terradue.com/hydro-smhi/description",
                    "gfdHydrogfdodQuery"="[hydrogfdod,SMHI,operational]",
                    "gfdHydrogfdodComment"="",
                            
                    "gfdOdDailySubDir"="od-daily",
                    "gfdOdDailyUrl"="https://catalog.terradue.com/hydro-smhi/description",
                    "gfdOdDailyQuery"="[od,daily,SMHI,operational]",
                    "gfdOdDailyComment"="",
                            
                    "gfdEcoperSubDir"="ecoper",
                    "gfdEcoperUrl"="https://catalog.terradue.com/hydro-smhi/description",
                    "gfdEcoperQuery"="[ecoper,SMHI,operational]",
                    "gfdEcoperComment"=""
  )

xCastsInterval <- list("hydrogfdeiStartDateSearch"="2019-06-01",
                       "hydrogfdeiEndDateSearch"="2019-06-30",
                       "hydrogfdeiStartDateFilename"="201906",
                       "hydrogfdeiEndDateFilename"="201906",
                       "hydrogfdodStartDateSearch"="2019-07-01",
                       "hydrogfdodEndDateSearch"="2019-08-31",
                       "hydrogfdodStartDateFilename"="201907",
                       "hydrogfdodEndDateFilename"="201908",
                       "odStartDateSearch"="2019-09-01",
                       "odEndDateSearch"="2019-10-02",
                       "odStartDateFilename"="20190901",
                       "odEndDateFilename"="20191002",
                       #"ecoperStartDateSearch"="2019-10-03",
                       #"ecoperEndDateSearch"="2019-10-12",
                       "ecoperStartDateSearch"="2019-08-26",
                       "ecoperEndDateSearch"="2019-06-26",
                       "ecoperStartDateFilename"="20191003",
                       "ecoperEndDateFilename"="20191012"
  )


In [232]:
urlNC     <- modelConfig$gfdHydrogfdeiUrl
catQuery  <- modelConfig$gfdHydrogfdeiQuery
startDate <- xCastsInterval$hydrogfdeiStartDateSearch
stopDate  <- xCastsInterval$hydrogfdeiEndDateSearch

netcdfDir <- paste(netcdfDirRoot,modelConfig$gfdHydrogfdeiSubDir,sep="/")

# Extend catQuery with variable name
print(catQuery)
variables <- c('pr','tas','tasmin','tasmax')

varcatQuery <- gsub(']',',pr]',catQuery)
varcatQuery

outvarSplit = trimws(strsplit(catQuery,split = ",")[[1]])
nOut=length(outvarSplit)
for(i in 1:nOut){
    #print(i)
    print(outvarSplit[i])
    
#    splitFirst = trimws(strsplit(outvarSplit[i],split = "(",fixed=T)[[1]])
#    oName=splitFirst[1]
#    splitSecond = trimws(strsplit(splitFirst[2],split = ")",fixed=T)[[1]])
#    oUnit=splitSecond[1]
#    oID=substr(splitSecond[2],nchar(splitSecond[2])-3,nchar(splitSecond[2]))
#    oName2=substr(splitSecond[2],1,nchar(splitSecond[2])-5)
#    if(i==1){
#        outvars.name=oName
#        outvars.name2=oName2
#        outvars.unit=oUnit
#        outvars.id=oID
#    }else{
#        outvars.name=c(outvars.name,oName)
#        outvars.name2=c(outvars.name2,oName2)
#        outvars.unit=c(outvars.unit,oUnit)
#        outvars.id=c(outvars.id,oID)
#    }
}
# Extend end

  opensearchCmd=paste0(osClientApp," '",urlNC,"'"," -p ","'count=unlimited'"," -p ","'cat=",catQuery,"'"," -p ","'start=",startDate,secStartDay,"'"," -p ","'stop=",stopDate,secEndDay,"'"," enclosure")
  message(opensearchCmd)
  input_enclosure <- system(command = opensearchCmd,intern = T)
  message(input_enclosure)
  if (length(input_enclosure >= 1)) {
    if (! dir.exists(netcdfDir)) { dir.create(netcdfDir) }
    for (url in 1:length(input_enclosure)) {
      rciop.copy(input_enclosure[url],netcdfDir)
    }
  }

# Looks like all files has been downloaded with start/stop in the same month interval.
#Test with an additional month? - 20190401-20190630 - ok, all 12 files.

[1] "[hydrogfdei,SMHI,operational]"


[1] "[hydrogfdei,SMHI,operational,pr]"

[1] "[hydrogfdei"
[1] "SMHI"
[1] "operational]"


opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[hydrogfdei,SMHI,operational]' -p 'start=2019-06-01T00:00:01' -p 'stop=2019-06-30T23:59:59' enclosure
https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/tas_hydrogfdei_201906_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/tasmin_hydrogfdei_201906_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/tasmax_hydrogfdei_201906_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/pr_hydrogfdei_201906_fanfar_SMHI.nc


[1] " -o /workspace/tmpdir/process-nc/hydrogfdei https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/tas_hydrogfdei_201906_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdei https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/tasmin_hydrogfdei_201906_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdei https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/tasmax_hydrogfdei_201906_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdei https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdei/2019/06/pr_hydrogfdei_201906_fanfar_SMHI.nc"


In [172]:
netcdfDir <- paste(netcdfDirRoot,modelConfig$gfdHydrogfdodSubDir,sep="/")

opensearchCmd=paste0(osClientApp," '",modelConfig$gfdHydrogfdodUrl,"'"," -p ","'count=unlimited'"," -p ","'cat=",modelConfig$gfdHydrogfdodQuery,"'"," -p ","'start=",xCastsInterval$hydrogfdodStartDateSearch,secStartDay,"'"," -p ","'stop=",xCastsInterval$hydrogfdodEndDateSearch,secEndDay,"'"," enclosure")
  message(opensearchCmd)
  input_enclosure <- system(command = opensearchCmd,intern = T)
  message(input_enclosure)
  if (length(input_enclosure >= 1)) {
    if (! dir.exists(netcdfDir)) { dir.create(netcdfDir) }
    for (url in 1:length(input_enclosure)) {
      rciop.copy(input_enclosure[url],netcdfDir)
    }
  }

#Test with an additional month? - 20190301-20190531 - ok, all 12 files.
#With the standard start/stop, this file seems to be missing pr_hydrogfdod_201908_fanfar_SMHI.nc


opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[hydrogfdod,SMHI,operational]' -p 'start=2019-07-01T00:00:01' -p 'stop=2019-08-31T23:59:59' enclosure
https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/08/tas_hydrogfdod_201908_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/08/tasmin_hydrogfdod_201908_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/08/tasmax_hydrogfdod_201908_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/tas_hydrogfdod_201907_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/tasmax_hydrogfdod_201907_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/tasmin_hydrogfdod_201907_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/pr_hydrogfdod_201907_fanfar_SMHI.nc


[1] " -o /workspace/tmpdir/process-nc/hydrogfdod https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/08/tas_hydrogfdod_201908_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdod https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/08/tasmin_hydrogfdod_201908_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdod https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/08/tasmax_hydrogfdod_201908_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdod https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/tas_hydrogfdod_201907_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdod https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/tasmax_hydrogfdod_201907_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/hydrogfdod https://store.terradue.com/hydro-smhi/fanfar/Operational/hydrogfdod/2019/07/tasmin_hydrogfdod_201907_fanfar_SMHI.nc"
[1] " -o /

In [214]:
netcdfDir <- paste(netcdfDirRoot,modelConfig$gfdOdDailySubDir,sep="/")
  
opensearchCmd=paste0(osClientApp," '",modelConfig$gfdOdDailyUrl,"'"," -p ","'count=unlimited'"," -p ","'cat=",modelConfig$gfdOdDailyQuery,"'"," -p ","'start=",xCastsInterval$odStartDateSearch,secStartDay,"'"," -p ","'stop=",xCastsInterval$odEndDateSearch,secEndDay,"'"," enclosure")
  message(opensearchCmd)
  input_enclosure <- system(command = opensearchCmd,intern = T)
  message(input_enclosure)
  if (length(input_enclosure >= 1)) {
    #nc_tmpdir <- paste(netcdfDir,"nc-files",sep="/")
    #if (! dir.exists(nc_tmpdir)) { dir.create(nc_tmpdir) }
    if (! dir.exists(netcdfDir)) { dir.create(netcdfDir) }
    for (url in 1:length(input_enclosure)) {
      #rciop.copy(input_enclosure[url],nc_tmpdir)
      rciop.copy(input_enclosure[url],netcdfDir)
    }
  }

# 2019-09-01 - 2019-10-02:
#Mostly ok, no files for october (maybe not uploaded to htep yet)
#There might also be one or two missing files in this range.

opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[od,daily,SMHI,operational]' -p 'start=2019-09-01T00:00:01' -p 'stop=2019-10-02T23:59:59' enclosure
https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmin_od-daily_20190929_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190929_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190929_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190928_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190928_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190927_fanfar_SMHI.nchttps://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmin_od-daily_20190927_fanfar_SMHI.nchttps://store.terradue.com

[1] " -o /workspace/tmpdir/process-nc/od-daily https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tasmin_od-daily_20190929_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/od-daily https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190929_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/od-daily https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190929_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/od-daily https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/tas_od-daily_20190928_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/od-daily https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190928_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/od-daily https://store.terradue.com/hydro-smhi/fanfar/Operational/od-daily/2019/09/pr_od-daily_20190927_fanfar_SMHI.nc"
[1] " -o /workspace/tmpdir/process-nc/od-daily

In [221]:
netcdfDir <- paste(netcdfDirRoot,modelConfig$gfdEcoperSubDir,sep="/")
  
opensearchCmd=paste0(osClientApp," '",modelConfig$gfdEcoperUrl,"'"," -p ","'count=unlimited'"," -p ","'cat=",modelConfig$gfdEcoperQuery,"'"," -p ","'start=",xCastsInterval$ecoperStartDateSearch,secStartDay,"'"," -p ","'stop=",xCastsInterval$ecoperEndDateSearch,secEndDay,"'"," enclosure")
  message(opensearchCmd)
  input_enclosure <- system(command = opensearchCmd,intern = T)
  message(input_enclosure)
  if (length(input_enclosure >= 1)) {
    #nc_tmpdir <- paste(netcdfDir,"nc-files",sep="/")
    #if (! dir.exists(nc_tmpdir)) { dir.create(nc_tmpdir) }
    if (! dir.exists(netcdfDir)) { dir.create(netcdfDir) }
    for (url in 1:length(input_enclosure)) {
      #rciop.copy(input_enclosure[url],nc_tmpdir)
      rciop.copy(input_enclosure[url],netcdfDir)
    }
  }

# Problem, query gives all files and from other years etc.
#
#opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[ecoper,SMHI,operational]' -p 'start=2019-10-03T00:00:01' -p 'stop=2019-10-12T23:59:59' enclosure
#opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[ecoper,SMHI,operational]' -p 'start=2019-03-25T00:00:01' -p 'stop=2019-04-12T23:59:59' enclosure
#
#https://store.terradue.com/hydro-smhi/fanfar/Operational/ecoper/2018/07/tasmin_ecoper_2018072900_fanfar_SMHI.nc
#https://store.terradue.com/hydro-smhi/fanfar/Operational/ecoper/2019/08/tasmax_ecoper_2019080200_fanfar_SMHI.nc
#...

opensearch-client 'https://catalog.terradue.com/hydro-smhi/description' -p 'count=unlimited' -p 'cat=[ecoper,SMHI,operational]' -p 'start=2019-08-26T00:00:01' -p 'stop=2019-06-26T23:59:59' enclosure



In [211]:
#
#
#
hydModel <- rciop.getparam('hydmodel')
hydModel


outvarSplit = trimws(strsplit(hydModel,split = ",")[[1]])
nOut=length(outvarSplit)
for(i in 1:nOut){
    print(i)
    print(outvarSplit[i])
    
    splitFirst = trimws(strsplit(outvarSplit[i],split = "(",fixed=T)[[1]])
    oName=splitFirst[1]
    splitSecond = trimws(strsplit(splitFirst[2],split = ")",fixed=T)[[1]])
    oUnit=splitSecond[1]
    oID=substr(splitSecond[2],nchar(splitSecond[2])-3,nchar(splitSecond[2]))
    oName2=substr(splitSecond[2],1,nchar(splitSecond[2])-5)
    if(i==1){
        outvars.name=oName
        outvars.name2=oName2
        outvars.unit=oUnit
        outvars.id=oID
    }else{
        outvars.name=c(outvars.name,oName)
        outvars.name2=c(outvars.name2,oName2)
        outvars.unit=c(outvars.unit,oUnit)
        outvars.id=c(outvars.id,oID)
    }
}


[1] "Niger-HYPE"      "WestAfrica-HYPE"

[1] 1
[1] "Niger-HYPE"
